In [404]:

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from pathlib import Path 
import sys 
import os 
import logging
import seaborn as sns
from decimal import Decimal, ROUND_DOWN

from scipy.stats import uniform, randint
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, GroupShuffleSplit
from sklearn.metrics import make_scorer ,root_mean_squared_error

from collections import Counter
from sklearn.preprocessing import FunctionTransformer, StandardScaler

from helper.transformers import CorrelationSelector,PreserveFeatureNames, PreserveFeatureNamesRegressor


from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GroupKFold,  cross_validate
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor 
from lightgbm import LGBMRegressor
from sklearn.svm import SVR

from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from skrub import TableVectorizer


from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, classification_report, confusion_matrix

sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict, audit, plot, save, ml


%load_ext autoreload
%autoreload 2

#shap.initjs()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [405]:
if Path('/Volumes/Samsung_T5/MIT/mta').exists():
    data_root =     '/Volumes/Samsung_T5/MIT/mta'
    data_derived  = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
else: 
    data_root = '/Users/alina/Desktop/MIT/code/data'
    data_derived  = '/Users/alina/Desktop/MIT/code/data/output/derived_data'

info_path = Path(data_root, "files")
types_file_path = Path(data_derived,"all_vars_description_ML.xlsx" )
#os.listdir(data_derived)


In [406]:
################## PREDICTORS ####################
pred = pd.read_csv(Path(data_derived, 'mta_data_clean.csv')).drop(columns = 'Unnamed: 0')
out = pd.read_csv(Path(data_derived, 'out_clean_all_raters.csv')).drop(columns = 'Unnamed: 0')


rater_out = 'm'
rater_pred = "m"

thr_drop_missing = 50 # if changed to lower number, some cols will be dropped, 50 iwas threshold in preprcessing (wont change anything)

model_type = "RF"
corr_select= False # use correlation selector or not 




In [407]:
outcome_short = "ODD"
file_name_save = 'results_ML_simple_CV_RF_XGB_{}.csv'.format(outcome_short)
file_path_save = Path(data_derived, 'ML_results', file_name_save)

In [408]:
out_dict = {"HYP": 'snap_snahypat', "INATT": 'snap_snainatt', "ODD": 'snap_snaoddt', 'SS': 'ssrs_ssptosst', "INTERN": 'ssrs_sspintt', "DOM" : 'pcrc_pcrcpax', 'INTIM': 'pcrc_pcrcprx'}
outcome_result_dict = {'ODD':  "SNAP ODD Symptoms T Score", "HYP": "SNAP Hyperactivity Symptoms T Score", "INATT" :"SNAP Inattention Symptoms T Score" , "INTERN": "SSRS Internalizing Symptoms", "SS": "SSRS Social Skills T Score", "DOM": "Parental Dominance Mean Score", "INTIM": "Parent-Child Intimacy Mean Score"}
outcome_var = out_dict[outcome_short]
outcome_var

'snap_snaoddt'

In [409]:
data, df_X, y, rater_count_X = ml.prepare_data(pred, out, rater_pred, rater_out, thr_drop_missing, outcome_var)

out rater Index(['src_subject_id', 'snap_snainatt_m', 'snap_snahypat_m',
       'snap_snaoddt_m', 'ssrs_sspintt_m', 'ssrs_ssptosst_m', 'pcrc_pcrcpax_m',
       'pcrc_pcrcprx_m'],
      dtype='object')
snap_snainatt_m (502,)
snap_snahypat_m (503,)
snap_snaoddt_m (503,)
ssrs_sspintt_m (497,)
ssrs_ssptosst_m (496,)
pcrc_pcrcpax_m (520,)
pcrc_pcrcprx_m (521,)
to predict :  snap_snaoddt_m_out
Index([], dtype='object')
Removing empty columns ..  N =  0
Index([], dtype='object')
(559, 348)
Removing constant columns .. N =  0
Index([], dtype='object')
(559, 348)
Removing known and raw columns..  N =  : 0
[]
(559, 348)
Removing above threshold empty columns.. N =  : 0
set()
(559, 348)
X shape  (503, 346)
Columns ending with 'm': 277
Columns ending with 'f': 0
Columns ending with 'c': 68
Columns ending with 't': 0


In [410]:
ml.check_duplicates(df_X)

Duplicated column names: []


[]

In [411]:

ord_vars, num_vars, cat_vars_str, cat_vars_num = ml.get_var_types(df_X, types_file_path)


Ordinal variables: 369
Numeric variables: 66
Categorical numeric variables: 12
Categorical string variables: 3
Overlap between num_vars and ord_vars: {'masc_ma31hfx_c', 'masc_ma22acx_c'}
Overlap between num_vars and cat_vars_str: set()
Overlap between num_vars and cat_vars_num: set()
Overlap between ord_vars and cat_vars_str: set()
Overlap between ord_vars and cat_vars_num: set()
Overlap between cat_vars_str and cat_vars_num: set()
Overlap across all lists: set()


In [412]:

scoring = {
    'r2': 'r2',
    'mse': 'neg_mean_squared_error',
    'mae': 'neg_mean_absolute_error'
}

groups = data['src_subject_id'].values

## Next   Compute 
- simple model results for XG boost , finish for RF
- accuracy as correlation prediction - label ? 
- add cross validation loop  
- feature importances in cross validation 
- split half reliabiliy of feature importances across splits 
- reliability of feature importances across models 

In [414]:
import warnings
warnings.filterwarnings("ignore")

if model_type == "XGB":
    regress =  XGBRegressor(random_state=42)
elif model_type == "RF":
    regress = RandomForestRegressor(random_state= 42)


num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')), # check paper 
    ('std_scaler',StandardScaler())
])

ord_pipe = Pipeline([
    ('imputer', PreserveFeatureNames(SimpleImputer(strategy='mean'))), # change that to OrdinalEncoder sklearn -- just encodes to catgoreies !!
    ('identity', PreserveFeatureNames(FunctionTransformer(lambda x: x)))
])


cat_str_pipe = Pipeline([
    ('imputer', PreserveFeatureNames(SimpleImputer(strategy='constant', fill_value='missing'))),
     ('ohe', PreserveFeatureNames(TableVectorizer()))
])

cat_num_pipe = Pipeline([
    ('imputer', PreserveFeatureNames(SimpleImputer(strategy='constant', fill_value=-1))),
    ('ohe', PreserveFeatureNames(OneHotEncoder(handle_unknown="ignore")))
])

preprocessor = ColumnTransformer([
    ('num', num_pipe, num_vars),
    ('cat_str', cat_str_pipe, cat_vars_str),
    ('cat_num', cat_num_pipe, cat_vars_num),
    ('ord', ord_pipe, ord_vars)
])

if corr_select : 
    pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('correlation_selector', CorrelationSelector(threshold=0.8)),
    ('regressor',regress)
    ])
else: 
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor',regress)
    ])

if model_type == "RF":
    param_distributions = {
        'regressor__n_estimators': [100, 300],          # Correct prefix
        'regressor__max_depth': [10, None],
        'regressor__min_samples_split': [2, 10],
        'regressor__min_samples_leaf': [1, 4]
    }
    
elif model_type == "XGB":
    param_distributions = {
        'regressor__n_estimators': [100, 300],
        'regressor__max_depth': [10],                    # Removed None
        'regressor__colsample_bytree': [0.5, 0.3],       # Mapped 'sqrt' and 'log2' to numeric values
        'regressor__min_child_weight': [2, 10, 1, 4],    # Combined 'min_samples_split' and 'min_samples_leaf'
    }
    
# Define the parameter grid

# Define the GroupKFold cross-validation
cv = GroupKFold(n_splits=5)

# Define the scorer
scorer = {
    'rmse': make_scorer(root_mean_squared_error, greater_is_better=False),
    'mae': make_scorer(mean_absolute_error, greater_is_better=False),
    'r2': make_scorer(r2_score)
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=10,  # Number of parameter settings to sample
    scoring=scorer,
    refit='rmse', 
    cv=cv,
    verbose=3,
    random_state=42
)

# Perform the search
random_search.fit(df_X, y, groups=groups)

# Print best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

#pipeline.fit(X_train_val, y_train_val)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__min_samples_split=2, regressor__n_estimators=100; mae: (test=-3.961) r2: (test=0.256) rmse: (test=-5.000) total time=   2.0s
[CV 2/5] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__min_samples_split=2, regressor__n_estimators=100; mae: (test=-3.742) r2: (test=0.320) rmse: (test=-4.821) total time=   1.9s
[CV 3/5] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__min_samples_split=2, regressor__n_estimators=100; mae: (test=-3.363) r2: (test=0.376) rmse: (test=-4.245) total time=   1.9s
[CV 4/5] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__min_samples_split=2, regressor__n_estimators=100; mae: (test=-3.865) r2: (test=0.243) rmse: (test=-5.033) total time=   1.9s
[CV 5/5] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__min_samples_split=2, regressor__n_estim

KeyboardInterrupt: 

In [ ]:
# best_pipeline = random_search.best_estimator_

# # 1) Get the list of final feature names from CorrelationSelector
# selector = best_pipeline.named_steps['correlation_selector']
# final_feature_names = selector.get_support(indices=False)  # returns the column names it kept

# # 2) Get the trained regressor and its feature importances
# final_model = best_pipeline.named_steps['regressor']
# importances = final_model.feature_importances_

# # 3) Pair up each feature name with its importance
# import pandas as pd

# feature_importance_df = pd.DataFrame({
#     'feature': final_feature_names,
#     'importance': importances
# }).sort_values(by='importance', ascending=False)

# print("\nFeature Importances:")
# print(feature_importance_df)

In [ ]:
# Assuming feat_imp_df is your DataFrame with 'feature' and 'importance' columns
# feat_imp_df = pd.DataFrame({
#     'feature': final_feature_names,
#     'importance': importances
# }).sort_values(by='importance', ascending=False)

# Plot the top 20 features
# top_n = 20
# top_features = feature_importance_df.head(top_n).sort_values(by='importance')

# plt.figure(figsize=(10, 8))
# plt.barh(top_features['feature'], top_features['importance'], color='skyblue')
# plt.xlabel('Feature Importance')
# plt.title(f'Top {top_n} Feature Importances')
# plt.tight_layout()
# plt.show()

In [ ]:
# y_pred = random_search.predict(X_test)
# # Calculate evaluation metrics
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"Mean Squared Error: {mse}")
# print(f"Root Mean Squared Error: {rmse}")
# print(f"Mean Absolute Error: {mae}")
# print(f"R^2 Score: {r2}")


In [ ]:
def truncate(value, decimals):
    value = Decimal(value)
    return float(value.quantize(Decimal(f"1.{'0' * decimals}"), rounding=ROUND_DOWN))


In [ ]:
outcome_var_table = "{} ({})".format(outcome_result_dict[outcome_short], rater_out.upper())
outcome_var_table

'SNAP ODD Symptoms T Score (M)'

In [ ]:
random_search.best_estimator_.named_steps.keys()

dict_keys(['preprocessor', 'regressor'])

- for r2 reuslts--> no need for center of mass approach 
- center of mass for feature importances (= rank sum vote?) 

In [ ]:
new_row = ml.get_results_from_random_search(random_search, outcome_short, thr_drop_missing)
new_row

{'m': 349, 'f': 0, 'c': 100, 't': 0}
Metrics for Best Parameters:
mean_test_rmse   -4.717573
mean_test_mae    -3.720533
mean_test_r2      0.302039
Name: 4, dtype: object


{'Model Name': 'RandomForestRegressor',
 'Cross Validation Type': 'GroupKFold(n_splits=5)',
 'Hyperparameters': 'n_estimators: 300\nmin_samples_split: 2\nmin_samples_leaf: 4\nmax_depth: None',
 'Mean Squared Error': 22.2554,
 'Root Mean Squared Error': 4.7175,
 'Mean Absolute Error': 3.7205,
 'R² Score': 0.302,
 'Outcome Variable': 'SNAP ODD Symptoms T Score',
 'Number of Features': 'Total: 450\nMother: 349\nFather: 0\nTeacher: 0\nChild: 100',
 'Feature Selection Method': 'No feature selection',
 'Threshold Drop Row': 50}

In [ ]:
new_row

{'Model Name': 'RandomForestRegressor',
 'Cross Validation Type': 'GroupKFold(n_splits=5)',
 'Hyperparameters': 'n_estimators: 300\nmin_samples_split: 2\nmin_samples_leaf: 4\nmax_depth: None',
 'Mean Squared Error': 22.2554,
 'Root Mean Squared Error': 4.7175,
 'Mean Absolute Error': 3.7205,
 'R² Score': 0.302,
 'Outcome Variable': 'SNAP ODD Symptoms T Score',
 'Number of Features': 'Total: 450\nMother: 349\nFather: 0\nTeacher: 0\nChild: 100',
 'Feature Selection Method': 'No feature selection',
 'Threshold Drop Row': 50}

In [ ]:
ml.save_cv_result_to_table(file_path_save, new_row, save=True, reduced= False, nrows2drop=2)

Column names from table :  ['Model Name', 'Cross Validation Type', 'Hyperparameters', 'Mean Squared Error', 'Root Mean Squared Error', 'Mean Absolute Error', 'R² Score', 'Outcome Variable', 'Number of Features', 'Feature Selection Method', 'Threshold Drop Row']
NEW RESULTS TABLE (not saved):
              Model Name   Cross Validation Type                                                                 Hyperparameters  Mean Squared Error  Root Mean Squared Error  Mean Absolute Error  R² Score           Outcome Variable                                          Number of Features Feature Selection Method  Threshold Drop Row
0  RandomForestRegressor  GroupKFold(n_splits=5)  n_estimators: 300\nmin_samples_split: 10\nmin_samples_leaf: 4\nmax_depth: None             34.0801                   5.8378               4.7928    0.0995  SNAP ODD Symptoms T Score  Total: 450\nMother: 349\nFather: 0\nTeacher: 0\nChild: 100     No feature selection                  50
1  RandomForestRegressor  GroupKF